<a href="https://colab.research.google.com/github/swilsonmfc/trees/blob/master/TicTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic Tac Toe

![](https://miro.medium.com/max/599/1*Wbif8jHo31boDYlbpDcT_Q@2x.jpeg)

# Setup

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import AdaBoostClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Data

In [33]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data

--2021-03-25 14:47:18--  https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25866 (25K) [application/x-httpd-php]
Saving to: ‘tic-tac-toe.data.1’

tic-tac-toe.data.1  100%[===================>]  25.26K   166KB/s    in 0.2s    

2021-03-25 14:47:19 (166 KB/s) - ‘tic-tac-toe.data.1’ saved [25866/25866]



In [34]:
names = [f'pos_{x}' for x in range(9)] + ['target']
df = pd.read_csv('tic-tac-toe.data', names=names)

In [35]:
df

,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,target
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,negative
954,o,x,o,x,x,o,x,o,x,negative
955,o,x,o,x,o,x,x,o,x,negative
956,o,x,o,o,x,x,x,o,x,negative


# Visualize

In [36]:
def draw_game(row):
  print(f"{row['pos_0']} {row['pos_1']} {row['pos_2']}")
  print(f"{row['pos_3']} {row['pos_4']} {row['pos_5']}")
  print(f"{row['pos_6']} {row['pos_7']} {row['pos_8']}")
  print(row['target'])

In [37]:
draw_game(df.iloc[0])

x x x
x o o
x o o
positive


In [38]:
draw_game(df.iloc[957])

o o x
x x o
o x x
negative


# Preprocess
* Encode x, o, b into numeric
* Convert positive / negative to binary

In [39]:
mapping_board  = {'o':0, 'x':1, 'b':2}
mapping_target = {'positive':1, 'negative':0}
for i in range(9):
  df[f'pos_{i}'] = df[f'pos_{i}'].map(mapping_board)
df['target']   = df['target'].map(mapping_target)

In [40]:
df

,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,target
0,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,0,0,0,1,0,1
2,1,1,1,1,0,0,0,0,1,1
3,1,1,1,1,0,0,0,2,2,1
4,1,1,1,1,0,0,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...
953,0,1,1,1,0,0,0,1,1,0
954,0,1,0,1,1,0,1,0,1,0
955,0,1,0,1,0,1,1,0,1,0
956,0,1,0,0,1,1,1,0,1,0


# Train Test

In [41]:
X = df.copy()
y = df['target']
X = X.drop(columns='target')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=151)

# Results

In [42]:
results_df = pd.DataFrame(columns=['Accuracy'])

# Model

In [43]:
ada = AdaBoostClassifier(random_state=15)
ada.fit(X_train, y_train)
pred = ada.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.56      0.69        85
           1       0.80      0.96      0.87       155

    accuracy                           0.82       240
   macro avg       0.84      0.76      0.78       240
weighted avg       0.83      0.82      0.81       240



## Lucky
* Did we get lucky with a split of the data?

In [44]:
ada = AdaBoostClassifier(random_state=125)
score = cross_val_score(ada, X, y).mean()
results_df.loc['Baseline'] = score
print(f'Score = {score:.4f}')

Score = 0.7328


# Improvements

## Increase Estimators - 100

In [45]:
ada = AdaBoostClassifier(n_estimators=100, random_state=125)
score = cross_val_score(ada, X, y).mean()
results_df.loc['100 Estimators'] = score
print(f'Score = {score:.4f}')

Score = 0.8006


## Increase Estimators - 200

In [46]:
ada = AdaBoostClassifier(n_estimators=200, random_state=125)
score = cross_val_score(ada, X, y).mean()
results_df.loc['200 Estimators'] = score
print(f'Score = {score:.4f}')

Score = 0.8904


## Increase Estimators - 500

In [47]:
ada = AdaBoostClassifier(n_estimators=500, random_state=125)
score = cross_val_score(ada, X, y).mean()
results_df.loc['500 Estimators'] = score
print(f'Score = {score:.4f}')

Score = 0.9645


## Learning Rate

In [48]:
ada = AdaBoostClassifier(n_estimators=1000, learning_rate=0.5, random_state=125)
score = cross_val_score(ada, X, y).mean()
results_df.loc['1000 Estimators - Smaller Learning Rate'] = score
print(f'Score = {score:.4f}')

Score = 0.9478


## Balance Classes

In [49]:
oversampler = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversampler.fit_resample(X, y)
ada = AdaBoostClassifier(n_estimators=500, random_state=125)
score = cross_val_score(ada, X_over, y_over).mean()
results_df.loc['500 Estimators - Oversampled'] = score
print(f'Score = {score:.4f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Score = 0.9489


# Feature Engineering

## Errors

In [50]:
ada = AdaBoostClassifier(n_estimators=500, random_state=125)
ada.fit(X_train, y_train)
pred = ada.predict(X_test)

err_df = X_test[~(y_test == pred)].copy()
err_df['target'] = y_test[~(y_test == pred)]
err_df['pred'] = pred[~(y_test == pred)]
err_df

,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,target,pred
957,0,0,1,1,1,0,0,1,1,0,1
945,1,0,1,1,1,0,0,1,0,0,1
954,0,1,0,1,1,0,1,0,1,0,1


In [51]:
for idx, row in err_df.iterrows():
  print('Example', idx)
  draw_game(row)

Example 957
0 0 1
1 1 0
0 1 1
0
Example 945
1 0 1
1 1 0
0 1 0
0
Example 954
0 1 0
1 1 0
1 0 1
0


## Board Positions

In [52]:
features = ['row_1', 'row_2', 'row_3',
            'col_1', 'col_2', 'col_3',
            'diag_1', 'diag_2']
colsets  = [
  ['pos_0', 'pos_1', 'pos_2'],
  ['pos_3', 'pos_4', 'pos_5'],
  ['pos_6', 'pos_7', 'pos_8'],

  ['pos_0', 'pos_3', 'pos_6'],
  ['pos_1', 'pos_4', 'pos_7'],
  ['pos_2', 'pos_5', 'pos_8'],

  ['pos_0', 'pos_4', 'pos_8'],
  ['pos_6', 'pos_4', 'pos_2']
]
markers = [0, 1]

def add_features(df):
  df_add = df.copy()
  for marker in markers:
    for colset, feature in zip(colsets, features):
      df_add[f'{feature}_{marker}'] = np.sum(df_add[colset] == marker, axis=1)
  return df_add

In [53]:
add_features(df)

,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,target,row_1_0,row_2_0,row_3_0,col_1_0,col_2_0,col_3_0,diag_1_0,diag_2_0,row_1_1,row_2_1,row_3_1,col_1_1,col_2_1,col_3_1,diag_1_1,diag_2_1
0,1,1,1,1,0,0,1,0,0,1,0,2,2,0,2,2,2,1,3,1,1,3,1,1,1,2
1,1,1,1,1,0,0,0,1,0,1,0,2,2,1,1,2,2,2,3,1,1,2,2,1,1,1
2,1,1,1,1,0,0,0,0,1,1,0,2,2,1,2,1,1,2,3,1,1,2,1,2,2,1
3,1,1,1,1,0,0,0,2,2,1,0,2,1,1,1,1,1,2,3,1,0,2,1,1,1,1
4,1,1,1,1,0,0,2,0,2,1,0,2,1,0,2,1,1,1,3,1,0,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,0,1,1,1,0,0,0,1,1,0,1,2,1,2,1,1,2,2,2,1,2,1,2,2,1,1
954,0,1,0,1,1,0,1,0,1,0,2,1,1,1,1,2,1,1,1,2,2,2,2,1,2,2
955,0,1,0,1,0,1,1,0,1,0,2,1,1,1,2,1,2,2,1,2,2,2,1,2,1,1
956,0,1,0,0,1,1,1,0,1,0,2,1,1,2,1,1,1,1,1,2,2,1,2,2,2,2


In [54]:
X_feature = add_features(X)
ada = AdaBoostClassifier(n_estimators=500, random_state=125)
score = cross_val_score(ada, X_feature, y).mean()
results_df.loc['Feature Engineering - Row Col Diag'] = score
print(f'Score = {score:.4f}')

Score = 0.9552


## Max Board Position

In [55]:
X_feature['MaxO'] = X_feature[[f'{feature}_0' for feature in features]].max(axis=1)
X_feature['MaxX'] = X_feature[[f'{feature}_1' for feature in features]].max(axis=1)

In [56]:
ada = AdaBoostClassifier(n_estimators=500, random_state=125)
score = cross_val_score(ada, X_feature[['MaxO', 'MaxX']], y).mean()
results_df.loc['Feature Engineering - Max X or Y'] = score
print(f'Score = {score:.4f}')

Score = 0.9832


# Results

In [57]:
results_df.sort_values('Accuracy', ascending=False)

,Accuracy
Feature Engineering - Max X or Y,0.983246
500 Estimators,0.964491
Feature Engineering - Row Col Diag,0.955208
500 Estimators - Oversampled,0.948880
1000 Estimators - Smaller Learning Rate,0.947818
200 Estimators,0.890412
100 Estimators,0.800649
Baseline,0.732777


## Unlucky
* Structure in the list of examples

In [58]:
ada = AdaBoostClassifier(n_estimators=500, random_state=125)
cross_val_score(ada, X_feature[['MaxO', 'MaxX']], y)

array([1.        , 1.        , 1.        , 1.        , 0.91623037])

In [59]:
idx = np.random.permutation(X_feature.index)
X_feature = X_feature.reindex(idx)
y         = y.reindex(idx)

ada = AdaBoostClassifier(n_estimators=500, random_state=125)
cross_val_score(ada, X_feature[['MaxO', 'MaxX']], y)

array([1., 1., 1., 1., 1.])